In [1]:
import pickle
from attention import *
from torch.utils.data import DataLoader
from pytorch_util import RAdam,trainable_parameter
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import TransformerDecoderLayer as TransformerDecoderLayer_torch
#from pytorch_transformers import AdamW,WarmupLinearSchedule

In [2]:
batch_size = 96
dim = 768
head_d = 8
head = SimplyInteraction_mol #SimplyHead
encoder_layer,decoder_layer = 3,3
epochs = 10
clip = 0.1
logLoss = True
EncoderLayer=TransformerEncoderLayer
DecoderLayer=TransformerDecoderLayer_torch

dim_feedforward = 1024
dropout = 0.05
lr = 5e-4

In [3]:
with open('../Data/train_atom_list.pickle', 'rb') as handle:
    train_atom_list=pickle.load(handle)
with open('../Data/train_coupling_list.pickle', 'rb') as handle:
    train_coupling_list=pickle.load(handle)    
with open('../Data/train_index_list.pickle', 'rb') as handle:
    train_index_list=pickle.load(handle)  
with open('../Data/train_target_list.pickle', 'rb') as handle:
    train_target_list=pickle.load(handle)      
    
with open('../Data/val_atom_list.pickle', 'rb') as handle:
    val_atom_list=pickle.load(handle)
with open('../Data/val_coupling_list.pickle', 'rb') as handle:
    val_coupling_list=pickle.load(handle)    
with open('../Data/val_index_list.pickle', 'rb') as handle:
    val_index_list=pickle.load(handle)  
with open('../Data/val_target_list.pickle', 'rb') as handle:
    val_target_list=pickle.load(handle)

# with open('../Data/test_atom_list.pickle', 'rb') as handle:
#     test_atom_list=pickle.load(handle)
# with open('../Data/test_coupling_list.pickle', 'rb') as handle:
#     test_coupling_list=pickle.load(handle)    
# with open('../Data/test_index_list.pickle', 'rb') as handle:
#     test_index_list=pickle.load(handle)  
# with open('../Data/test_target_list.pickle', 'rb') as handle:
#     test_target_list=pickle.load(handle)

In [4]:
train_dl = attentionDataset2(train_atom_list,train_coupling_list,train_index_list,train_target_list)
train_dl = DataLoader(train_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn4,num_workers=4)
val_dl = attentionDataset2(val_atom_list,val_coupling_list,val_index_list,val_target_list)
val_dl = DataLoader(val_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn4,num_workers=4)

In [25]:
# %debug
# atom,atom_mask,coupling,coupling_mask,index_,target = next(iter(train_dl))

In [9]:
model=Attention_mol(dim,encoder_layer,decoder_layer,head_d,head,EncoderLayer,DecoderLayer,
                 dropout=dropout,dim_feedforward=dim_feedforward).to('cuda')
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr,weight_decay=1e-2)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-04)
model, opt = amp.initialize(model, opt, opt_level="O1")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [6]:
# out,mask,edge,y = next(iter(train_dl))
# out,mask,edge,y = out.to('cuda:0'),mask.to('cuda:0'),edge.to('cuda:0'),y.to('cuda:0')

# opt.zero_grad()
# loss,loss_perType = model(out,mask,edge,y,logLoss)
# loss.backward()

# [(n,p.std()) for n,p in model.named_parameters()]

# [(n,p.grad.mean()) for n,p in model.named_parameters()]

In [10]:
model,bestOpt,bestWeight = train_type_mol(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

epoch:0, train_loss: +0.965, val_loss: +0.531, 
train_vector: +2.45|+1.85|+0.77|+0.80|+0.46|+0.77|+0.82|-0.20, 
val_vector  : +1.59|+1.19|+0.51|+0.51|+0.07|+0.50|+0.33|-0.44

epoch:1, train_loss: +0.448, val_loss: +0.145, 
train_vector: +1.93|+1.13|+0.42|+0.35|-0.20|+0.41|+0.13|-0.59, 
val_vector  : +1.38|+0.85|+0.22|-0.02|-0.55|+0.24|-0.13|-0.84

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
epoch:2, train_loss: +0.192, val_loss: -0.087, 
train_vector: +1.66|+0.88|+0.22|+0.03|-0.49|+0.22|-0.14|-0.85, 
val_vector  : +1.01|+0.62|+0.06|-0.28|-0.71|+0.05|-0.40|-1.04

epoch:3, train_loss: -0.042, val_loss: -0.155, 
train_vector: +1.21|+0.71|+0.04|-0.24|-0.69|+0.04|-0.35|-1.05, 
val_vector  : +0.92|+0.92|-0.05|-0.47|-0.79|-0.03|-0.55|-1.19

epoch:4, train_loss: -0.193, val_loss: -0.341, 
train_vector: +1.01|+0.60|-0.10|-0.41|-0.84|-0.11|-0.51|-1.19, 
val_vector  : +0.79|+0.47|-0.23|-0.54|-1.01|-0.24|-0.64|-1.32

Gradient overflow.  Skipping step, loss scale

In [8]:
torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict()
            }, '../Model/attention_edge_512_4_10.tar')

In [ ]:
checkpoint = torch.load('../Model/attention_edge_512_4_10.tar')
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])